In [1]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import legacy
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
dataset_dir = 'private/tomatect'
base_dir = 'private/splits'
train_pct = 0.8
validation_pct = 0.15
test_pct = 0.05
img_width = 150
img_height = 150
batch_size = 32

In [3]:
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir)

In [4]:
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    
    train_class_dir = os.path.join(train_dir, class_name)
    os.makedirs(train_class_dir)
    
    validation_class_dir = os.path.join(validation_dir, class_name)
    os.makedirs(validation_class_dir)
    
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(test_class_dir)

In [5]:
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    
    train_class_dir = os.path.join(train_dir, class_name)
    validation_class_dir = os.path.join(validation_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    
    all_files = os.listdir(class_dir)
    num_files = len(all_files)
    
    num_train = int(num_files * train_pct)
    num_validation = int(num_files * validation_pct)
    num_test = int(num_files * test_pct)
    
    random.shuffle(all_files)
    
    train_files = all_files[:num_train]
    validation_files = all_files[num_train:num_train+num_validation]
    test_files = all_files[-num_test:]
    
    for file_name in train_files:
        src_file = os.path.join(class_dir, file_name)
        dst_file = os.path.join(train_class_dir, file_name)
        shutil.copyfile(src_file, dst_file)
    
    for file_name in validation_files:
        src_file = os.path.join(class_dir, file_name)
        dst_file = os.path.join(validation_class_dir, file_name)
        shutil.copyfile(src_file, dst_file)
    
    for file_name in test_files:
        src_file = os.path.join(class_dir, file_name)
        dst_file = os.path.join(test_class_dir, file_name)
        shutil.copyfile(src_file, dst_file)

In [32]:
dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    shuffle=True,
    image_size=(img_height,img_width),
    batch_size=batch_size
)

dataset_validation = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    seed=123,
    shuffle=True,
    image_size=(img_height,img_width),
    batch_size=batch_size
)


dataset_test = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    shuffle=False,
    image_size=(img_height,img_width),
    batch_size=batch_size
)

Found 183 files belonging to 5 classes.
Found 30 files belonging to 5 classes.
Found 9 files belonging to 5 classes.


In [33]:
class_names = dataset_train.class_names

dataset_train = dataset_train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
dataset_validation = dataset_validation.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [34]:
data_scaling = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(img_height, img_width),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
   tf.keras.layers.experimental.preprocessing.RandomRotation(0.9),
   tf.keras.layers.experimental.preprocessing.RandomContrast(0.5),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
  tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
  tf.keras.layers.experimental.preprocessing.RandomWidth(0.2)    
])

In [35]:
train_ds = dataset_train.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)


In [36]:
input_shape = (batch_size, img_width, img_height, 3)
n_classes = 10

In [37]:
model = tf.keras.models.Sequential([
    data_scaling,
  #  data_augmentation,
    tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(filters=16,  kernel_size = (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(filters=16,  kernel_size = (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout((0.2)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(n_classes, activation='softmax'),
])

In [38]:
model.build(input_shape=input_shape)

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_8 (Sequential)   (32, 150, 150, 3)         0         
                                                                 
 conv2d_15 (Conv2D)          (32, 148, 148, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPooli  (32, 74, 74, 32)          0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (32, 72, 72, 16)          4624      
                                                                 
 max_pooling2d_16 (MaxPooli  (32, 36, 36, 16)          0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (32, 34, 34, 16)        

In [39]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [40]:
history = model.fit(
    dataset_train,
    batch_size=32,
    #steps_per_epoch=len(dataset_test)// batch_size,
    validation_data=dataset_validation,
    verbose=1,
    epochs=25,
)

Epoch 1/25


2023-06-06 20:36:23.048004: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [183]
	 [[{{node Placeholder/_4}}]]
2023-06-06 20:36:23.048405: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [183]
	 [[{{node Placeholder/_0}}]]
2023-06-06 20:36:23.307053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - ETA: 0s - loss: 1.9113 - accuracy: 0.1639

2023-06-06 20:36:26.300749: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [30]
	 [[{{node Placeholder/_4}}]]
2023-06-06 20:36:26.301109: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [30]
	 [[{{node Placeholder/_4}}]]
2023-06-06 20:36:26.356648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 4s 231ms/step - loss: 1.9113 - accuracy: 0.1639 - val_loss: 1.6551 - val_accuracy: 0.2000
Epoch 2/25
6/6 [==============================] - 0s 62ms/step - loss: 1.6072 - accuracy: 0.2350 - val_loss: 1.6307 - val_accuracy: 0.2000
Epoch 3/25
6/6 [==============================] - 0s 87ms/step - loss: 1.4718 - accuracy: 0.3716 - val_loss: 1.5399 - val_accuracy: 0.2667
Epoch 4/25
6/6 [==============================] - 0s 83ms/step - loss: 1.2846 - accuracy: 0.4863 - val_loss: 1.7272 - val_accuracy: 0.2333
Epoch 5/25
6/6 [==============================] - 0s 79ms/step - loss: 1.2243 - accuracy: 0.4973 - val_loss: 1.5441 - val_accuracy: 0.4000
Epoch 6/25
6/6 [==============================] - 1s 93ms/step - loss: 1.0352 - accuracy: 0.5956 - val_loss: 1.7202 - val_accuracy: 0.2000
Epoch 7/25
6/6 [==============================] - 0s 79ms/step - loss: 0.9625 - accuracy: 0.6612 - val_loss: 1.5475 - val_accuracy: 0.3333
Epoch 8/25
6/6 [=====================